In [2]:
import findspark
findspark.init()

In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark import SparkContext

In [9]:
pyspark = SparkSession.builder \
.master("local[4]") \
.appName("Create a RDD") \
.config("spark.executor.memory", "4g") \
.config("spark.driver.memory", "2g") \
.getOrCreate()

## 1. RDD Basic Transformations

In [10]:
sc = pyspark.sparkContext

In [11]:
my_list = [1,2,3,4,5,6,7,8,8,4,42,67]

RDD yaratmak için parallelize

In [12]:
list_rdd = sc.parallelize(my_list)

In [13]:
list_rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 8, 4]

### map()

Klasik mapping

In [14]:
list_rdd.map(lambda x: x*x*x).take(5)

[1, 8, 27, 64, 125]

### filter()

Koşula göre filtre

In [15]:
list_rdd.filter(lambda x: x == 8).take(5)

[8, 8]

### flatMap()

Her elemanı mapleme

In [16]:
txt = ["Hello", "How are you?","CERN-LHC", "Data Engineering"]

In [17]:
txt_rdd = sc.parallelize(txt)

In [18]:
txt_rdd.flatMap(lambda x: x.split(" ")).map(lambda x: x.upper()).take(9)

['HELLO', 'HOW', 'ARE', 'YOU?', 'CERN-LHC', 'DATA', 'ENGINEERING']

### distinct()

RDD içindeki unique elemanları seçer

In [19]:
list_rdd.distinct().take(10)

[1, 2, 3, 67, 4, 5, 6, 7, 8, 42]

### sample()

RDD'lerden seed ile sample seçimi

In [20]:
list_rdd.sample(True, 0.7, 45).take(10)

[2, 2, 3, 4, 5, 8, 8, 4, 42, 42]

## 2. RDD Pair Transformations

In [21]:
my_list_1 = [1,5,7,9,5,7]
my_list_2 = [1,4,8,19]
rdd_my_list_1 = sc.parallelize(my_list_1)
rdd_my_list_2 = sc.parallelize(my_list_2)

### union()

RDD'lerin birleşimini verir

In [22]:
rdd_my_list_1.union(rdd_my_list_2).take(10)

[1, 5, 7, 9, 5, 7, 1, 4, 8, 19]

### intersection()

RDD'lerin kesişimini verir

In [23]:
rdd_my_list_1.intersection(rdd_my_list_2).take(10)

[1]

### substract()

RDD'lerin farkını alır

In [24]:
rdd_my_list_1.subtract(rdd_my_list_2).take(10)

[5, 5, 7, 7, 9]

### cartesian()

2 RDD ile cartezyen ikilisi oluşturma

In [25]:
rdd_my_list_1.cartesian(rdd_my_list_2).take(10)

[(1, 1),
 (1, 4),
 (1, 8),
 (1, 19),
 (5, 1),
 (5, 4),
 (5, 8),
 (5, 19),
 (7, 1),
 (7, 4)]

## 3. RDD Basic Actions

### collect()

RDD'deki tüm elemanları getirir

In [26]:
rdd_my_list_1.collect()

[1, 5, 7, 9, 5, 7]

### count() 

Length of RDD

In [27]:
rdd_my_list_1.count()

6

### countByValue()

RDD içinde her bir değerden kaç tane varsa onun ile dict oluşturup getirir

In [28]:
rdd_my_list_1.countByValue()

defaultdict(int, {1: 1, 5: 2, 7: 2, 9: 1})

### take()

RDD içinden istenilen kadar eleman getirme

In [29]:
rdd_my_list_1.take(3)

[1, 5, 7]

### top()

En üstten istenilen kadar eleman getirme

In [30]:
rdd_my_list_1.top(4)

[9, 7, 7, 5]

### takeOrdered()

İstenilen kadar elemanı sıralı getirme

In [31]:
rdd_my_list_1.takeOrdered(20)

[1, 5, 5, 7, 7, 9]

### takeSample()

istenilen kadar sample'ı seede kadar take ile eager getirir

In [32]:
rdd_my_list_1.takeSample(False, 5, 33)

[1, 7, 9, 7, 5]

### reduce()

Elemanları kurala göre tek elemana indirger

In [33]:
rdd_my_list_1.reduce(lambda x,y: x+y)

34

### fold()

reduce'a çok benzer

In [34]:
rdd_my_list_1.fold(0, lambda x,y: x+y)

34

### aggregate()

sırayla lambda fonksiyonlarını uygular. Bu örnekte elemanları toplayarak tek elemana indirger ve length verir

In [41]:
print(rdd_my_list_1.collect())
rdd_my_list_1.aggregate((0,0), lambda x, y: (x[0] + y, x[1] + 1), lambda x, y: (x[0] + y[0], x[1] + y[1]))


[1, 5, 7, 9, 5, 7]


(34, 6)